In [ ]:
import pandas as pd
import numpy as np

In [ ]:
texasCasesDataURL = "https://dshs.texas.gov/coronavirus/TexasCOVID-19NewCasesOverTimebyCounty.xlsx"
texasDeathsDataURL = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"
texasTestingLegacyDataUrl = "https://dshs.texas.gov/coronavirus/TexasCOVID-19CumulativeTestsOverTimebyCounty.xlsx"
texasTestingDataUrl = "https://dshs.texas.gov/coronavirus/TexasCOVID-19CumulativeTestsbyCounty.xlsx"
texasCaseCountDataUrl = "https://dshs.texas.gov/coronavirus/TexasCOVID19CaseCountData.xlsx"

In [ ]:
casesDf = pd.read_excel(texasCasesDataURL, skiprows=2)
deathsDf = pd.read_excel(texasDeathsDataURL, skiprows=2, skipfooter=2)
legacytestsDf = pd.read_excel(texasTestingLegacyDataUrl, skiprows=1, skipfooter=11)
testscasesDf = pd.read_excel(texasTestingDataUrl, skiprows=1, skipfooter=3)

In [ ]:
headers = casesDf.columns
headers = [i.split(" ")[2] for i in headers[1:]]
temp = []
for i in headers:
    if "2020-" not in i:
        temp.append("2020-" + i)
    else:
        temp.append(i)
headers = temp
headers.insert(0, 'County')
casesDf.columns = headers

In [ ]:
casesDfMelted = casesDf.melt(id_vars='County', var_name='Date', value_name='Cases')
casesDfMelted.Date = pd.to_datetime(casesDfMelted.Date)

In [ ]:
headers = deathsDf.columns
headers = [i.split(' ')[1] for i in headers]
headers = ['2020-' + i for i in headers[1:]]
headers.insert(0, 'County')
deathsDf.columns = headers

In [ ]:
deathsDfMelted = deathsDf.melt(id_vars='County', var_name='Date', value_name='Deaths')
deathsDfMelted.Date = pd.to_datetime(casesDfMelted.Date)
deathsDfMelted.County = deathsDfMelted.County.str.capitalize()

In [ ]:
headers = legacytestsDf.columns
headers = [i.split(' ')[2:4] for i in headers]
headers = [' '.join(i) for i in headers]
headers = [i + ', 2020' for i in headers]
headers = [i.replace('*', '') for i in headers]
headers = [pd.to_datetime(i) for i in headers[1:]]
headers = [i.strftime("%Y-%m-%d") for i in headers]
headers.insert(0, 'County')
legacytestsDf.columns = headers

In [ ]:
headers = testscasesDf.columns
headers = [i.strftime("%Y-%m-%d") for i in headers[1:]]
headers.insert(0, 'County')
testscasesDf.columns = headers

In [ ]:
testsDf = pd.concat([legacytestsDf, testscasesDf], axis=1)
testsDf = testsDf.loc[:,~testsDf.columns.duplicated()]
testsDf.drop(columns=['2020-05-05', '2020-05-23', '2020-06-10'], inplace=True)

In [ ]:
testsDfMelted = testsDf.melt(id_vars='County', var_name='Date', value_name='Tests')
testsDfMelted.Date = pd.to_datetime(testsDfMelted.Date)

In [ ]:
unifiedTexasDf = pd.merge(testsDfMelted, casesDfMelted)
unifiedTexasDf = pd.merge(unifiedTexasDf, deathsDfMelted)
unifiedTexasDf.Tests = unifiedTexasDf.Tests.astype('int')

In [ ]:
unifiedTexasDf

In [ ]:
graphDf = unifiedTexasDf.copy()
graphDf.set_index('Date', inplace=True)
graphDf.groupby('County')['Tests'].plot(legend=True)

In [ ]:
texasCasesDf = pd.merge(casesDfMelted.groupby('Date').sum().reindex(), deathsDfMelted.groupby('Date').sum().reindex(), on = "Date", how = "outer").ffill() 
texasCasesDf

In [ ]:
texasCaseCountDf = pd.read_excel(texasCaseCountDataUrl, skiprows=2, skipfooter=14, usecols="A:C", sheet_name="Tests by Day")
texasCaseCountDf["Negative"] = texasCaseCountDf["Test Results"] - texasCaseCountDf["Positive Test Results"]
texasCaseCountDf["State"] = "Texas"
texasCaseCountDf.drop(["Test Results"], axis=1, inplace=True)
texasCaseCountDf.rename(columns={"Lab Reported Date" : "Date", "Positive Test Results" : "Positive"}, inplace=True)
texasCaseCountDf

In [ ]:
texasCaseCountDfCopy = texasCaseCountDf.copy()

In [ ]:
texasCaseCountDfCopy = texasCaseCountDfCopy.merge(texasCasesDf, on = "Date", how = "outer").dropna()
texasCaseCountDfCopy["State"] = "Texas"
texasCaseCountDf = texasCaseCountDfCopy[["Date", "State", "Cases", "Positive", "Negative", "Deaths"]]
texasCaseCountDf.dtypes

In [ ]:
texasCaseCountDf.__dict__.update(texasCaseCountDf.astype({"Positive": np.int64, "Negative": np.int64, "Deaths" : np.int64}).__dict__)
texasCaseCountDf = texasCaseCountDf.iloc[::-1]
texasCaseCountDf.to_csv('../data/processed/by_state/TX.csv', index=False)

In [ ]:
byStateDf = unifiedTexasDf.copy()
byStateDf["State"] = "Texas"
byStateDf = byStateDf[["Date","State","County","Cases","Deaths"]]
byStateDf.to_csv('../data/processed/by_county/TX.csv', index=False)